In [1]:
import re
from sentence_transformers import SentenceTransformer

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
from sentence_transformers import CrossEncoder

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd

In [3]:
df = pd.read_json("tables_3.json")

In [4]:
df.head()

,table,description,param,class
0,bha_run,Журнал работы бурового инструмента (BHA),"{'well_id': 'ID скважины', 'interval_from/to':...",Техпараметры бурения
1,auto_drilling_program_stage,Этапы автоматизированной программы бурения,{'depth_from/to': 'Глубина начала/окончания эт...,Управление бурением
2,auto_drilling_program,Программы автоматического бурения,"{'well_id': 'ID скважины', 'depth_from/to': 'Ц...",Управление бурением
3,drilling_parameter,Параметры бурения в реальном времени,{'rop_average': 'Средняя скорость проходки (м/...,Техпараметры бурения
4,operation,Журнал операций по скважине,"{'operation_type': 'Тип операции (бурение, спу...",Мониторинг


In [4]:
def replace_slash(text):
    return str(text).replace('/', ' ')

In [5]:
def params_to_str(params):
    if isinstance(params, dict):
        param_str = ' '.join([f"{k} {v}" for k, v in params.items()])  # Ключ и значение через пробел
        return replace_slash(param_str)  # Заменяем слэши
    return replace_slash(str(params))

In [7]:
params_to_str(df.param[0])

'well_id ID скважины interval_from to Интервал бурения (м) chisel_type size Тип размер долота nnd_min max avg Мин макс средняя нагрузка на долото (тс) pressure_min max Давление (атм) time_duration_hours Продолжительность работы (часы)'

In [6]:
df['combined_text'] = (
    df['description'].str.lower() + ' ' +
    df['param'].apply(params_to_str).str.lower() + ' ' +
    df['class'].str.lower()
)

In [11]:
df.head()

,table,description,param,class,combined_text
0,bha_run,Журнал работы бурового инструмента (BHA),"{'well_id': 'ID скважины', 'interval_from/to':...",Техпараметры бурения,журнал работы бурового инструмента (bha) well_...
1,auto_drilling_program_stage,Этапы автоматизированной программы бурения,{'depth_from/to': 'Глубина начала/окончания эт...,Управление бурением,этапы автоматизированной программы бурения dep...
2,auto_drilling_program,Программы автоматического бурения,"{'well_id': 'ID скважины', 'depth_from/to': 'Ц...",Управление бурением,программы автоматического бурения well_id id с...
3,drilling_parameter,Параметры бурения в реальном времени,{'rop_average': 'Средняя скорость проходки (м/...,Техпараметры бурения,параметры бурения в реальном времени rop_avera...
4,operation,Журнал операций по скважине,"{'operation_type': 'Тип операции (бурение, спу...",Мониторинг,журнал операций по скважине operation_type тип...


In [9]:
df.combined_text[0]

'журнал работы бурового инструмента (bha) well_id id скважины interval_from to интервал бурения (м) chisel_type size тип размер долота nnd_min max avg мин макс средняя нагрузка на долото (тс) pressure_min max давление (атм) time_duration_hours продолжительность работы (часы) техпараметры бурения'

In [50]:
reranker_model = CrossEncoder(
    model_name="DiTy/cross-encoder-russian-msmarco",
    max_length=512)

In [7]:
query = "какие шаблонные размеры имеет лист документа"

In [28]:
candidates = [
    "Позиция была создана в 1849 году, за год до того, как Калифорния стала штатом. Нынешний губернатор - Джерри Браун, демократ, вступивший в должность 3 января 2011 года и ранее занимавший пост губернатора с 1975 по 1983 год. В октябре 2013 года Браун превзошел графа Уоррена по самому продолжительному совокупному периоду времени, когда он занимал пост губернатора. Из 38 губернаторов, занимавших этот пост, семь из них фактически родились в Калифорнии (шесть из них после обретения статуса штата): 2 Один (Ромуальдо Пачеко) родился в Санта-Барбаре. 3 Четверо (Джордж Парди, Джеймс Рольф, Пэт Браун и Джерри Браун) родились в Сан-Франциско.",
    "Исполняющий обязанности губернатора - это конституционная должность, созданная в некоторых штатах США, когда губернатор умирает при исполнении служебных обязанностей или уходит в отставку. В некоторых штатах губернатор может быть объявлен недееспособным и неспособным работать по разным причинам, включая болезнь и отсутствие в штате более определенного периода. Исполняющий обязанности губернатора исполняет обязанности до следующих всеобщих выборов, пока губернатор не выздоровеет. от его / ее травм, или, если губернатор умирает / уходит в отставку менее чем за 16 месяцев до окончания срока, до конца срока.",
    'Каждый лист документа должен иметь поля: левое – не менее 25 мм, правое – 15−20 мм, верхнее и нижнее – не менее 20 мм. Документы печатаются, как правило, на одной стороне листа. При печатании документа на оборотной стороне листа размер левого поля должен быть 15–20 мм, правого – не менее 25 мм, верхнего и нижнего – не менее 20 мм.',
    "На основе описей дел постоянного хранения и дел по личному составу составляются в соответствии с рекомендациями и требованиями соответствующего архивного учреждения годовые разделы соответствующих сводных описей дел аппарата управления общества, а также филиала и структурного подразделения, относящихся к источникам комплектования Архивного фонда Российской Федерации и заключивших с архивным учреждением договор о сотрудничестве (приложение № 16).",
    "Входящие документы, не требующие рассмотрения руководителем общества (его заместителями), направляются работником службы делопроизводства непосредственно в подразделения общества.",
    "Если в поступившем документе имеется ссылка на другие документы, отдел документационного обеспечения или служба делопроизводства соответствующего подразделения прилагает их (их копии) к поступившему документу.",
]

In [8]:
candidates = df.combined_text.tolist()

In [7]:
query = "Выведи данные о всех пользователях"

In [64]:
rank_result = reranker_model.rank(query, candidates[0:20])

In [65]:
rank_result

[{'corpus_id': 18, 'score': 0.0041263895},
 {'corpus_id': 19, 'score': 0.002956605},
 {'corpus_id': 3, 'score': 0.0027691661},
 {'corpus_id': 6, 'score': 0.0022125598},
 {'corpus_id': 8, 'score': 0.0021909778},
 {'corpus_id': 16, 'score': 0.0014262607},
 {'corpus_id': 0, 'score': 0.0013529216},
 {'corpus_id': 4, 'score': 0.0013425458},
 {'corpus_id': 12, 'score': 0.0013191095},
 {'corpus_id': 1, 'score': 0.0011970515},
 {'corpus_id': 9, 'score': 0.0011938985},
 {'corpus_id': 10, 'score': 0.0011808786},
 {'corpus_id': 14, 'score': 0.0011798189},
 {'corpus_id': 13, 'score': 0.0011794363},
 {'corpus_id': 5, 'score': 0.0011696541},
 {'corpus_id': 17, 'score': 0.0011261913},
 {'corpus_id': 15, 'score': 0.0010985039},
 {'corpus_id': 11, 'score': 0.0010840694},
 {'corpus_id': 2, 'score': 0.0010823255},
 {'corpus_id': 7, 'score': 0.0010739246}]

In [39]:
candidates[134]

'пользователи системы username логин role_old роль (устаревшее) organization организация управление доступом'

In [41]:
df.iloc[157]

table                                                        tasks
description                                 Фоновые задачи системы
param            {'payload': 'Данные задачи (JSON)', 'status': ...
class                                                Автоматизация
combined_text    фоновые задачи системы payload данные задачи (...
Name: 157, dtype: object

In [56]:
model = SentenceTransformer(
    model_name_or_path="DiTy/bi-encoder-russian-msmarco")

In [57]:
import torch


def normalize_embeddings(emb):
    return torch.nn.functional.normalize(torch.Tensor(emb).unsqueeze(0), p=2, dim=1)


def cos_sim(a, b):
    a_norm = normalize_embeddings(a)
    b_norm = normalize_embeddings(b)

    return torch.mm(a_norm, b_norm.transpose(0, 1)).item()


def semantic_search(query, candidates, top_k=1):
    assert candidates.shape[0] >= top_k, "Error, count candidates < top_k value"
    scores = []

    for candidate in candidates:
        # считаем косинусную близость по каждому кандидату
        scores.append(cos_sim(query, candidate))
    # выбираем только top-k кандидатов
    values, ids = torch.topk(torch.Tensor(scores), k=top_k, sorted=True)

    return [{'id': id_, 'score': value_} for id_, value_ in zip(ids.cpu().tolist(), values.cpu().tolist())]

In [58]:
# преобразуем кандидатов в векторы
embeddings = model.encode(candidates)
# преобразуем запрос в вектор
query_embeddings = model.encode(query)

In [59]:
results = semantic_search(query_embeddings, embeddings, top_k=5)

In [60]:
results

[{'id': 151, 'score': 0.22965764999389648},
 {'id': 149, 'score': 0.1542915403842926},
 {'id': 20, 'score': 0.14054036140441895},
 {'id': 133, 'score': 0.12165237963199615},
 {'id': 97, 'score': 0.09457278251647949}]

In [63]:
df.iloc[20]

table                                      work_of_special_technic
description                                     Работа спецтехники
param            {'vehicle_name': 'Тип техники', 'quantity': 'К...
class                                                    Логистика
combined_text    работа спецтехники vehicle_name тип техники qu...
Name: 20, dtype: object

In [9]:
from vectorizer import create_table_embeddings, find_relevant_tables

In [10]:
table_embeddings = create_table_embeddings(candidates)

In [11]:
relevant_tables = find_relevant_tables(query, table_embeddings, 40)

In [21]:
relevant_tables[10:]

[74,
 49,
 92,
 138,
 144,
 106,
 68,
 152,
 148,
 38,
 67,
 133,
 71,
 19,
 9,
 46,
 87,
 10,
 125,
 2,
 8,
 130,
 111,
 108,
 134,
 88,
 15,
 141,
 158,
 56]

In [23]:
df.iloc[133]

table                                      work_of_special_technic
description                          Работа спецтехники по проекту
param            {'vehicle_name': 'Тип техники', 'total_quantit...
class                                                    Логистика
combined_text    работа спецтехники по проекту vehicle_name тип...
Name: 133, dtype: object

In [20]:
df[130:140]

,table,description,param,class,combined_text
130,project_knbk,Проектные КНБК,"{'outer_diameter': 'Наружный диаметр (мм)', 'r...",Инструмент,проектные кнбк outer_diameter наружный диаметр...
131,project_liquors_plan,Параметры буровых растворов,"{'density_min/max': 'Плотность (г/см³)', 'filt...",Буровые растворы,параметры буровых растворов density_min max пл...
132,project_pressure_gradients_by_section,Градиенты давления по интервалам,"{'interval_from/to': 'Интервал (м)', 'fracture...",Геомеханика,градиенты давления по интервалам interval_from...
133,work_of_special_technic,Работа спецтехники по проекту,"{'vehicle_name': 'Тип техники', 'total_quantit...",Логистика,работа спецтехники по проекту vehicle_name тип...
134,users,Пользователи системы,"{'username': 'Логин', 'role_old': 'Роль (устар...",Управление доступом,пользователи системы username логин role_old р...
135,permissions,Права доступа,"{'code': 'Код права', 'group_title': 'Группа п...",Управление доступом,права доступа code код права group_title групп...
136,user_permissions,Назначение прав пользователям,"{'user_id': 'ID пользователя', 'permission_cod...",Управление доступом,назначение прав пользователям user_id id польз...
137,user_settings,Персональные настройки,"{'user_id': 'ID пользователя', 'data': 'Данные...",Конфигурация,персональные настройки user_id id пользователя...
138,nci_casing_column_names,Наименования обсадных колонн,"{'value': 'Название (кондуктор, эксплуатационн...",Конструкция скважин,наименования обсадных колонн value название (к...
139,nci_work_types,Типы работ,"{'key': 'Код', 'value': 'Наименование', 'is_sh...",Классификатор,типы работ key код value наименование is_show_...
